In [48]:
import numpy as np
from scipy.special import gamma
# import matplotlib.pyplot as plt
# import seaborn as sns

In [49]:
np.random.seed(42)

T_c = np.random.normal(loc=10, scale=3, size=10)
T_k = np.random.normal(loc=10, scale=3, size=5)
t_ck = np.random.normal(loc=10, scale=3, size=(10,5))

T_c[6] = T_c[6] * 0.5

theta = 3
rho = 0.4
alpha = 7
eta = 1.65

w_ck = np.ones((10,5))
P_yk = np.ones(5)
Y_c = np.ones(10)

In [50]:
P = np.sum(P_yk)
Y = np.sum(Y_c)
lambda_k = np.sum(t_ck * w_ck, axis=0)
Gamma = gamma((theta - 1) / theta)

# top = P_yk ** (alpha - eta)
# bottom = lambda_k ** (- rho)

# left = (np.outer(T_c, T_k) ** (alpha - 1)) / t_ck
# middle = top / bottom
# middle = np.tile(middle, (10, 1))
# right = (P ** eta) * Y / Gamma

# w_ck = (left * middle * right) ** (1 / (theta + alpha - 1))
# z_ck = Gamma * t_ck * (w_ck ** (theta - 1)) * (lambda_k ** (- rho))
# Q_ck = np.outer(T_c, T_k) * z_ck
# P_yk = np.mean(w_ck / np.outer(T_c, T_k), axis=1)
# Y = np.sum(Q_ck)

# Initialize variables
epsilon = 1e-6
Y = np.sum(Y_c)
Y_prev = Y + 2 * epsilon  # Ensure the loop starts
Y_values = []  # List to store Y values for each iteration

while abs(Y - Y_prev) >= epsilon:
    Y_prev = Y
    P = np.sum(P_yk)
    Y = np.sum(Y_c)
    lambda_k = np.sum(t_ck * w_ck, axis=0)

    top = P_yk ** (alpha - eta)
    bottom = lambda_k ** (- rho)

    left = (np.outer(T_c, T_k) ** (alpha - 1)) / t_ck
    middle = top / bottom
    middle = np.tile(middle, (10, 1))
    right = (P ** eta) * Y / Gamma

    w_ck = (left * middle * right) ** (1 / (theta + alpha - 1))
    z_ck = Gamma * t_ck * (w_ck ** (theta - 1)) * (lambda_k ** (- rho))
    Q_ck = np.outer(T_c, T_k) * z_ck
    P_yk = np.mean(w_ck / np.outer(T_c, T_k), axis=0)
    Y = np.sum(Q_ck)
    
    Y_values.append(Y)  # Append the current value of Y to the list

pi_ck_left = t_ck * (w_ck ** theta)
pi_ck_right = (lambda_k ** (- rho)) / np.sum(lambda_k ** (1 - rho))
pi_ck = pi_ck_left * pi_ck_right

pi_c_left = (t_ck * (w_ck ** theta)) / np.sum(t_ck * (w_ck ** theta), axis=0)
pi_c_right = (lambda_k ** (1 - rho)) / np.sum(lambda_k ** (1 - rho))
pi_c = np.sum(pi_c_left * pi_c_right, axis=1)

In [51]:
# np.savetxt('params/T_c.csv', T_c, delimiter=',')
# np.savetxt('params/T_k.csv', T_k, delimiter=',')
# np.savetxt('params/t_ck.csv', t_ck, delimiter=',')
# np.savetxt('output/w_ck.csv', w_ck, delimiter=',')
# np.savetxt('output/P_yk.csv', P_yk, delimiter=',')
# np.savetxt('output/Q_ck.csv', Q_ck, delimiter=',')
np.savetxt('output/pi_ck_shock.csv', pi_ck, delimiter=',')
# np.savetxt('output/pi_c_shock.csv', pi_c, delimiter=',')